# Debug, Evaluate, and Monitor LLMs with Client

LangChain makes it easy to prototype LLM applications and Agents. Even so, delivering a high-quality product to production can be deceptively difficult. You will likely have to heavily customize your prompts, chains, and other components to create a high-quality product.

To aid the development process, we've designed tracing and callbacks at the core of LangChain. In this notebook, you will get started prototyping, testing, and monitoring an example LLM agent.

When might you want to use tracing? Some situations we've found it useful include:
- Visualize how components relate
- Quickly debug a new chain, agent, or set of tools
- Evaluate a given different prompts and LLMs for a single component
- Running a given chain multiple time on a dataset to ensure it consistently meets a quality bar.
- Capturing usage traces and using LLMs or analytics pipelines to generate insights

## Prerequisites

**Run the local tracing server OR [create a hosted LangSmith account](https://smith.langchain.com/) and connect with an API key.**

To run the local server, execute the following comand in your terminal:
```
pip install --upgrade langsmith
langsmith start
```

Now, let's get started debugging!

## Debug your Chain 

First, configure your environment variables to tell LangChain to log traces. This is done by setting the `LANGCHAIN_TRACING_V2` environment variable to true.
You can tell LangChain which project to log to by setting the `LANGCHAIN_PROJECT` environment variable. This will automatically create a debug project for you.

For more information on other ways to set up tracing, please reference the [LangSmith documentation](https://docs.smith.langchain.com/docs/)

**NOTE:** You must also set your `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables in order to run the following tutorial.

**NOTE:** You can optionally set the `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables if using the hosted version.

In [1]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # Uncomment this line to use the hosted version
# os.environ["LANGCHAIN_API_KEY"] = "<YOUR-LANGSMITH-API-KEY>"  # Uncomment this line to use the hosted version.

# Used by the agent in this tutorial
# os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
# os.environ["SERPAPI_API_KEY"] = "<YOUR-SERPAPI-API-KEY>"

In [2]:
from langsmith import Client

client = Client()
print("You can click the link below to view the UI")
client

You can click the link below to view the UI


Client (API URL: https://dev.api.smith.langchain.com)

Now, start prototyping your agent. We will use a math example using an older ReACT-style agent.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [4]:
import asyncio

inputs = [
    "How many people live in canada as of 2023?",
    "who is dua lipa's boyfriend? what is his age raised to the .43 power?",
    "what is dua lipa's boyfriend age raised to the .43 power?",
    "how far is it from paris to boston in miles",
    "what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?",
    "what was the total number of points scored in the 2023 super bowl raised to the .23 power?",
    "how many more points were scored in the 2023 super bowl than in the 2022 super bowl?",
    "what is 153 raised to .1312 power?",
    "who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?",
    "what is 1213 divided by 4345?",
]
results = []


async def arun(agent, input_example):
    try:
        return await agent.arun(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


for input_example in inputs:
    results.append(arun(agent, input_example))
results = await asyncio.gather(*results)

In [5]:
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

# Logs are submitted in a background thread to avoid blocking execution.
# For the sake of this tutorial, we want to make sure
# they've been submitted before moving on. This is also
# useful for serverless deployments.
wait_for_all_tracers()

Assuming you've successfully configured the server earlier, your agent traces should show up in your server's UI. You can check by clicking on the link below:

In [6]:
client

Client (API URL: https://dev.api.smith.langchain.com)

## Test

Once you've debugged a customized your LLM component, you will want to create tests and benchmark evaluations to measure its performance before putting it into a production environment.

In this notebook, you will run evaluators to test an agent. You will do so in a few steps:

1. Create a dataset
2. Select or create evaluators to measure performance
3. Define the LLM or Chain initializer to test
4. Run the chain and evaluators using the helper functions

### 1. Create Dataset

Below, use the client to create a dataset from the Agent runs you just logged while debugging above. You will use these later to measure performance.

For more information on datasets, including how to create them from CSVs or other files or how to create them in the web app, please refer to the [LangSmith documentation](https://docs.langchain.plus/docs).

In [7]:
dataset_name = f"calculator-example-dataset-{unique_id}"

dataset = client.create_dataset(
    dataset_name, description="A calculator example dataset"
)

runs = client.list_runs(
    project_name=os.environ["LANGCHAIN_PROJECT"],
    execution_order=1,  # Only return the top-level runs
    error=False,  # Only runs that succeed
)
for run in runs:
    client.create_example(inputs=run.inputs, outputs=run.outputs, dataset_id=dataset.id)

### 2. Select RunEvaluators


**Note: the feedback API is currently experimental and subject to change.**

### 1. Define the Agent or LLM to Test

You can evaluate any LLM or chain. Since chains can have memory, we will pass in a `chain_factory` (aka a `constructor` ) function to initialize for each call.

In this case, you will test an agent that uses OpenAI's function calling endpoints, but it can be any simple chain.

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Since chains can be stateful (e.g. they can have memory), we provide
# a way to initialize a new chain for each row in the dataset. This is done
# by passing in a factory function that returns a new chain for each row.
def agent_factory():
    return initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=False)


# If your chain is NOT stateful, your factory can return the object directly
# to improve runtime performance. For example:
# chain_factory = lambda: agent

## 2. Configure Evaluation

Manually comparing the results of chains in the UI is effective, but it can be time consuming.
It can be helpful to use automated metrics and ai-assisted feedback to evaluate your component's performance.

Below, we will create some pre-implemented run evaluators that do the following:
- Compare results against ground truth labels. (You used the debug outputs above for this)
- Measure semantic (dis)similarity using embedding distance
- Evaluate 'aspects' of the agent's response in a reference-free manner using custom criteria

For a longer discussion of how to select an appropriate evaluator for your use case and how to create your own
custom evaluators, please refer to the [LangSmith documentation](https://docs.langchain.plus/docs/).


In [9]:
from langchain.evaluation import EvaluatorType
from langchain.langsmith import RunEvalConfig

evaluation_config = RunEvalConfig(
    # Evaluators can either be an evaluator type (e.g., "qa", "criteria", "embedding_distance", etc.) or a configuration for that evaluator
    evaluators=[
        EvaluatorType.QA,  # "Correctness" against a reference answer
        EvaluatorType.EMBEDDING_DISTANCE,
        RunEvalConfig.Criteria("helpfulness"),
        RunEvalConfig.Criteria(
            {
                "fifth-grader-score": "Do you have to be smarter than a fifth grader to answer this question?"
            }
        ),
    ]
)

### 3. Run the Agent and Evaluators

Use the `arun_on_dataset` (or synchronous `run_on_dataset`) function to evaluate your model. This will:
1. Fetch example rows from the specified dataset
2. Run your llm or chain on each example.
3. Apply evalutors to the resulting run traces and corresponding reference examples to generate automated feedback.

The results will be visible in the LangSmith app.

In [10]:
from langchain.client import (
    arun_on_dataset,
    run_on_dataset,  # Available if your chain doesn't support async calls.
)

chain_results = await arun_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_factory,
    evaluation=evaluation_config,
    verbose=True,
    tags=["testing-notebook"],  # Optional, adds a tag to the resulting chain runs
)

# Sometimes, the agent will error due to parsing issues, incompatible tool inputs, etc.
# These are logged as warnings here and captured as errors in the tracing UI.

Chain failed for example da00ffeb-0e60-4c38-8258-d42fc41bae0d. Error: LLMMathChain._evaluate("
age_of_Dua_Lipa_boyfriend ** 0.43
") raised error: 'age_of_Dua_Lipa_boyfriend'. Please try again with a valid numerical expression


Chain failed for example df2bb6d5-872f-4327-ba9f-bed0b2cd1626. Error: Too many arguments to single-input tool Calculator. Args: ['height ^ 0.13', {'height': 68}]


In [11]:
# For more information on additional configuration for the evaluation function:

?arun_on_dataset

Signature:
arun_on_dataset(
    dataset_name: 'str',
    llm_or_chain_factory: 'MODEL_OR_CHAIN_FACTORY',
    *,
    evaluation: 'Optional[RunEvalConfig]' = None,
    concurrency_level: 'int' = 5,
    num_repetitions: 'int' = 1,
    project_name: 'Optional[str]' = None,
    verbose: 'bool' = False,
    client: 'Optional[Client]' = None,
    tags: 'Optional[List[str]]' = None,
    input_mapper: 'Optional[Callable[[Dict], Any]]' = None,
) -> 'Dict[str, Any]'
Docstring:
Asynchronously run the Chain or language model on a dataset
and store traces to the specified project name.

Args:
    dataset_name: Name of the dataset to run the chain on.
    llm_or_chain_factory: Language model or Chain constructor to run
        over the dataset. The Chain constructor is used to permit
        independent calls on each example without carrying over state.
    concurrency_level: The number of async tasks to run concurrently.
    num_repetitions: Number of times to run the model on each example.
        

### Review the Test Results

You can review the test results tracing UI below by navigating to the "Datasets & Testing" page and selecting the **"calculator-example-dataset-*"** dataset and associated test project.

This will show the new runs and the feedback logged from the selected evaluators.

In [12]:
# You can navigate to the UI by clicking on the link below
client

Client (API URL: https://dev.api.smith.langchain.com)

For a real production application, you will want to add many more test cases and
incorporate larger datasets to compare aggregate performance. For more information, see the [LangSmith documentation](https://docs.langchain.plus/docs/)

## Monitor

Once your agent passed the selected quality bar, you can deploy it to production. For this notebook, you will simulate user interactions directly while logging your traces to Client for monitoring.

For more information on real production deployments, check out the [LangChain documentation](https://python.langchain.com/docs/guides/deployments/) or contact us at [support@langchain.dev](mailto:support@langchain.dev).

**First, create a new project to use in your production deployment.**

In [13]:
deployment_name = f"Search + Calculator Deployment - {unique_id}"
project = client.create_project(deployment_name)

**Then, deploy your agent to production, making sure to configure the environment to log to the monitoring project.**

In [14]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=False)

In [15]:
os.environ["LANGCHAIN_PROJECT"] = deployment_name

inputs = [
    "What's the ratio of the current US GDP to the average lifespan of a human?",
    "What's sin of 180 degrees?",
    "I need help on my homework",
    "If the price of bushel of wheat increases by 10 cents, about how much will that impact the average cost of bread?",
    # etc.
]
for query in inputs:
    try:
        await agent.arun(query)
    except Exception as e:
        print(e)

## Conclusion

Congratulations! You have succesfully traced and evaluated an agent using LangSmith!

This was a quick guide to get started, but there are many more ways to use LangSmith to speed up your developer flow and produce better results.

For more information on how you can get the most out of LangSmith, check out [LangSmith documentation](https://docs.langchain.plus/docs/), and please reach out with questions, feature requests, or feedback at [support@langchain.dev](mailto:support@langchain.dev).